In [1]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/macos/anaconda3/envs/dl/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [5]:
AutoModelForCausalLM.from_pretrained?

In [7]:
Trainer?

In [ ]:
@classmethod
def from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs):
    config = kwargs.pop("config", None)
    trust_remote_code = kwargs.pop("trust_remote_code", None)
    kwargs["_from_auto"] = True
    hub_kwargs_names = [
        "cache_dir",
        "force_download",
        "local_files_only",
        "proxies",
        "resume_download",
        "revision",
        "subfolder",
        "use_auth_token",
        "token",
    ]
    hub_kwargs = {name: kwargs.pop(name) for name in hub_kwargs_names if name in kwargs}
    code_revision = kwargs.pop("code_revision", None)
    commit_hash = kwargs.pop("_commit_hash", None)
    adapter_kwargs = kwargs.pop("adapter_kwargs", None)

    token = hub_kwargs.pop("token", None)
    use_auth_token = hub_kwargs.pop("use_auth_token", None)
    if use_auth_token is not None:
        warnings.warn(
            "The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.",
            FutureWarning,
        )
        if token is not None:
            raise ValueError(
                "`token` and `use_auth_token` are both specified. Please set only the argument `token`."
            )
        token = use_auth_token

    if token is not None:
        hub_kwargs["token"] = token

    if commit_hash is None:
        if not isinstance(config, PretrainedConfig):
            # We make a call to the config file first (which may be absent) to get the commit hash as soon as possible
            resolved_config_file = cached_file(
                pretrained_model_name_or_path,
                CONFIG_NAME,
                _raise_exceptions_for_gated_repo=False,
                _raise_exceptions_for_missing_entries=False,
                _raise_exceptions_for_connection_errors=False,
                **hub_kwargs,
            )
            commit_hash = extract_commit_hash(resolved_config_file, commit_hash)
        else:
            commit_hash = getattr(config, "_commit_hash", None)

    if is_peft_available():
        if adapter_kwargs is None:
            adapter_kwargs = {}
            if token is not None:
                adapter_kwargs["token"] = token

        maybe_adapter_path = find_adapter_config_file(
            pretrained_model_name_or_path, _commit_hash=commit_hash, **adapter_kwargs
        )

        if maybe_adapter_path is not None:
            with open(maybe_adapter_path, "r", encoding="utf-8") as f:
                adapter_config = json.load(f)

                adapter_kwargs["_adapter_model_path"] = pretrained_model_name_or_path
                pretrained_model_name_or_path = adapter_config["base_model_name_or_path"]

    if not isinstance(config, PretrainedConfig):
        kwargs_orig = copy.deepcopy(kwargs)
        # ensure not to pollute the config object with torch_dtype="auto" - since it's
        # meaningless in the context of the config object - torch.dtype values are acceptable
        if kwargs.get("torch_dtype", None) == "auto":
            _ = kwargs.pop("torch_dtype")
        # to not overwrite the quantization_config if config has a quantization_config
        if kwargs.get("quantization_config", None) is not None:
            _ = kwargs.pop("quantization_config")

        config, kwargs = AutoConfig.from_pretrained(
            pretrained_model_name_or_path,
            return_unused_kwargs=True,
            trust_remote_code=trust_remote_code,
            code_revision=code_revision,
            _commit_hash=commit_hash,
            **hub_kwargs,
            **kwargs,
        )

        # if torch_dtype=auto was passed here, ensure to pass it on
        if kwargs_orig.get("torch_dtype", None) == "auto":
            kwargs["torch_dtype"] = "auto"
        if kwargs_orig.get("quantization_config", None) is not None:
            kwargs["quantization_config"] = kwargs_orig["quantization_config"]

    has_remote_code = hasattr(config, "auto_map") and cls.__name__ in config.auto_map
    has_local_code = type(config) in cls._model_mapping.keys()
    trust_remote_code = resolve_trust_remote_code(
        trust_remote_code, pretrained_model_name_or_path, has_local_code, has_remote_code
    )

    # Set the adapter kwargs
    kwargs["adapter_kwargs"] = adapter_kwargs

    if has_remote_code and trust_remote_code:
        class_ref = config.auto_map[cls.__name__]
        model_class = get_class_from_dynamic_module(
            class_ref, pretrained_model_name_or_path, code_revision=code_revision, **hub_kwargs, **kwargs
        )
        _ = hub_kwargs.pop("code_revision", None)
        if os.path.isdir(pretrained_model_name_or_path):
            model_class.register_for_auto_class(cls.__name__)
        else:
            cls.register(config.__class__, model_class, exist_ok=True)
        return model_class.from_pretrained(
            pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs, **kwargs
        )
    elif type(config) in cls._model_mapping.keys():
        model_class = _get_model_class(config, cls._model_mapping)
        return model_class.from_pretrained(
            pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs, **kwargs
        )
    raise ValueError(
        f"Unrecognized configuration class {config.__class__} for this kind of AutoModel: {cls.__name__}.\n"
        f"Model type should be one of {', '.join(c.__name__ for c in cls._model_mapping.keys())}."
    )

In [6]:
%cat ~/anaconda3/envs/dl/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py

# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Factory function to build auto-model classes."""

import copy
import importlib
import json
import os
import warnings
from collections import OrderedDict

from ...configuration_utils import PretrainedConfig
from ...dynamic_module_utils import get_class_from_dynamic_module, resolve_trust_remote_code
from ...utils import (
    CONFIG_NAME,
    cached_file,
    copy_func,
    extract_commit_ha

In [ ]:
def train(
    Main training entry point.
            of [`Trainer`]. If present, training will resume from the model/optimizer/scheduler states loaded here.
            gathering predictions for evaluation during the training.
    self.is_in_train = True
    # do_train is not a reliable argument, as it might not be set and .train() still called, so
    if (args.fp16_full_eval or args.bf16_full_eval) and not args.do_train:
        raise TypeError(f"train() received got unexpected keyword arguments: {', '.join(list(kwargs.keys()))}.")
    self._train_batch_size = self.args.train_batch_size
        # In case of repeating the find_executable_batch_size, set `self._train_batch_size` properly
        if state.train_batch_size is not None:
            self._train_batch_size = state.train_batch_size
    inner_training_loop = find_executable_batch_size(
        self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size
            return inner_training_loop(
        return inner_training_loop(
def _inner_training_loop(
    self._train_batch_size = batch_size
    if self.state.train_batch_size != self._train_batch_size:
        # Temporarily unset `self.args.train_batch_size`
        original_bs = self.args.per_device_train_batch_size
        self.args.per_device_train_batch_size = self._train_batch_size // max(1, self.args.n_gpu)
        self.args.per_device_train_batch_size = original_bs
    self.state.train_batch_size = self._train_batch_size
logger.debug(f"Currently training with a batch size of: {self._train_batch_size}")
# Data loader and number of training steps
train_dataloader = self.get_train_dataloader()
    train_dataloader = tpu_spmd_dataloader(train_dataloader)
# Setting up training control variables:
# number of training epochs: num_train_epochs
# number of training steps per epoch: num_update_steps_per_epoch
# total number of training steps to execute: max_steps
total_train_batch_size = self._train_batch_size * args.gradient_accumulation_steps * args.world_size
num_train_tokens = None
if has_length(train_dataloader):
    len_dataloader = len(train_dataloader)
    num_examples = self.num_examples(train_dataloader)
        num_train_epochs = args.max_steps // num_update_steps_per_epoch + int(
        # May be slightly incorrect if the last batch in the training dataloader has a smaller size but it's
        num_train_samples = args.max_steps * total_train_batch_size
            num_train_tokens = (
                self.num_tokens(train_dataloader, args.max_steps) * args.gradient_accumulation_steps
        max_steps = math.ceil(args.num_train_epochs * num_update_steps_per_epoch)
        num_train_epochs = math.ceil(args.num_train_epochs)
        num_train_samples = self.num_examples(train_dataloader) * args.num_train_epochs
            num_train_tokens = self.num_tokens(train_dataloader) * args.num_train_epochs
    num_train_epochs = sys.maxsize
    num_examples = total_train_batch_size * args.max_steps
    num_train_samples = args.max_steps * total_train_batch_size
        num_train_tokens = self.num_tokens(train_dataloader, args.max_steps) * args.gradient_accumulation_steps
    self.optimizer, self.lr_scheduler = deepspeed_init(self, num_training_steps=max_steps)
    self.create_optimizer_and_scheduler(num_training_steps=max_steps)
self.state.train_batch_size = self._train_batch_size
    self.create_optimizer_and_scheduler(num_training_steps=max_steps)
    self.model.train()
logger.info("***** Running training *****")
logger.info(f"  Num Epochs = {num_train_epochs:,}")
logger.info(f"  Instantaneous batch size per device = {self.args.per_device_train_batch_size:,}")
if self.args.per_device_train_batch_size != self._train_batch_size:
    logger.info(f"  Training with DataParallel so batch size has been adjusted to: {self._train_batch_size:,}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_train_batch_size:,}")
logger.info(f"  Number of trainable parameters = {get_model_param_count(model, trainable_only=True):,}")
epochs_trained = 0
steps_trained_in_current_epoch = 0
steps_trained_progress_bar = None
# Check if continuing training from a checkpoint
    self.compare_trainer_and_checkpoint_args(self.args, self.state)
    epochs_trained = self.state.global_step // num_update_steps_per_epoch
        steps_trained_in_current_epoch = self.state.global_step % (num_update_steps_per_epoch)
        steps_trained_in_current_epoch *= args.gradient_accumulation_steps
        steps_trained_in_current_epoch = 0
    logger.info("  Continuing training from checkpoint, will skip to saved global_step")
    logger.info(f"  Continuing training from epoch {epochs_trained}")
    logger.info(f"  Continuing training from global step {self.state.global_step}")
            f"  Will skip the first {epochs_trained} epochs then the first"
            f" {steps_trained_in_current_epoch} batches in the first epoch."
self.callback_handler.train_dataloader = train_dataloader
# This should be the same if the state has been saved but in case the training arguments changed, it's safer
self.state.num_train_epochs = num_train_epochs
self.control = self.callback_handler.on_train_begin(args, self.state, self.control)
for epoch in range(epochs_trained, num_train_epochs):
    epoch_iterator = train_dataloader
    if epoch == epochs_trained and resume_from_checkpoint is not None and steps_trained_in_current_epoch == 0:
    if steps_trained_in_current_epoch > 0:
        epoch_iterator = skip_first_batches(epoch_iterator, steps_trained_in_current_epoch)
        steps_skipped = steps_trained_in_current_epoch
        steps_trained_in_current_epoch = 0
        # Skip past any already trained steps if resuming training
        if steps_trained_in_current_epoch > 0:
            steps_trained_in_current_epoch -= 1
            if steps_trained_progress_bar is not None:
                steps_trained_progress_bar.update(1)
            if steps_trained_in_current_epoch == 0:
        elif steps_trained_progress_bar is not None:
            steps_trained_progress_bar.close()
            steps_trained_progress_bar = None
            tr_loss_step = self.training_step(model, inputs)
        if self.control.should_epoch_stop or self.control.should_training_stop:
            "There seems to be not a single sample in your epoch_iterator, stopping training at step"
        self.control.should_training_stop = True
                "configured. Check your training configuration if this is unexpected."
    if self.control.should_training_stop:
    # Clean the state at the end of training
train_loss = self._total_loss_scalar / effective_global_step
    "train",
    num_samples=num_train_samples,
    num_tokens=num_train_tokens,
metrics["train_loss"] = train_loss
self.is_in_train = False
self.control = self.callback_handler.on_train_end(args, self.state, self.control)
# After training we make sure to retrieve back the original forward pass method
return TrainOutput(self.state.global_step, train_loss, metrics)

            import ray.train
            run_id = ray.train.get_context().get_trial_id()
        config = PretrainedConfig.from_json_file(config_file)
                f"You are resuming training from a checkpoint trained with {checkpoint_version} of "
        # If train a model using PEFT & LoRA, assume that adapter have been saved properly.
                        model.load_adapter(peft_id, subdir_name, is_trainable=(subdir_name == active_adapter))
                    model.load_adapter(resume_from_checkpoint, active_adapter, is_trainable=True)
                # If train a model using PEFT & LoRA, assume that adapter have been saved properly.
            f"Could not locate the best model at {best_model_path}, if you are running a distributed training "
                f"The `metric_for_best_model` training argument is set to '{metric_to_check}', which is not found in the evaluation metrics. "
                f"Didn't find an RNG file for process {process_index}, if you are resuming a training that "
                "Didn't find an RNG file, if you are resuming a training that was launched in a distributed "
                    "\nThis won't yield the same results as if the training had not been interrupted."
                    "\nThis won't yield the same results as if the training had not been interrupted."
                    "\nThis won't yield the same results as if the training had not been interrupted."
                f"The `metric_for_best_model` training argument is set to '{metric_to_check}', which is not found in the evaluation metrics. "
    # Save RNG state in non-distributed training
                # We use the CPU when training on one GPU to avoid OOM for GPU RAM when training big models.
                # In distributed training however, we load directly on each GPU and risk the GPU OOM as it's more
            logger.info("Continuing training from checkpoint, restoring any callbacks that were passed in")
            [`~trainer_utils.default_hp_space_optuna`] or [`~trainer_utils.default_hp_space_ray`] or
            [`~trainer_utils.default_hp_space_sigopt`] depending on your backend.
            method. Will default to [`~trainer_utils.default_compute_objective`].
        backend (`str` or [`~training_utils.HPSearchBackend`], *optional*):
        [`trainer_utils.BestRun` or `List[trainer_utils.BestRun]`]: All the information about the best run or best
    Log `logs` on the various objects watching training.
            "The batch received was empty, your model won't be able to train on it. Double-check that your "
            f"training dataset contains keys expected by the model: {','.join(self._signature_columns)}."

In [8]:
%cat ~/anaconda3/envs/dl/lib/python3.8/site-packages/transformers/trainer.py | grep 'train'

The Trainer class, to easily train a 🤗 Transformers from scratch or finetune it on a new task.
from .configuration_utils import PretrainedConfig
from .trainer_callback import (
from .trainer_pt_utils import (
from .trainer_utils import (
from .training_args import OptimizerNames, ParallelMode, TrainingArguments
    from .trainer_pt_utils import smp_forward_backward, smp_forward_only, smp_gather, smp_nested_concat
TRAINING_ARGS_NAME = "training_args.bin"
TRAINER_STATE_NAME = "trainer_state.json"
    Trainer is a simple but feature-complete training and eval loop for PyTorch, optimized for 🤗 Transformers.
            The model to train, evaluate or use for predictions. If not provided, a `model_init` must be passed.
            The arguments to tweak for training. Will default to a basic instance of [`TrainingArguments`] with the
            `output_dir` set to a directory named *tmp_trainer* in the current directory if not provided.
            The function to use to form a batch from a